In [2]:
import os
os.chdir('/content')
CODE_DIR = 'higan'
!git clone https://github.com/genforce/higan.git $CODE_DIR
os.chdir(f'./{CODE_DIR}')
!mkdir -p models/pretrain/pytorch
!wget https://www.dropbox.com/s/h1w7ld4hsvte5zf/stylegan_bedroom256_generator.pth?dl=1 -O models/pretrain/pytorch/stylegan_bedroom256_generator.pth --quiet
!wget https://www.dropbox.com/s/hwjyclj749qtp89/order_w.npy?dl=1 -O order_w_1k.npy --quiet

Cloning into 'higan'...
remote: Enumerating objects: 288, done.
remote: Counting objects: 100% (288/288), done.
remote: Compressing objects: 100% (226/226), done.
remote: Total 288 (delta 77), reused 263 (delta 58), pack-reused 0 (from 0)
Receiving objects: 100% (288/288), 16.22 MiB | 10.31 MiB/s, done.
Resolving deltas: 100% (77/77), done.


In [3]:
!pip install ttach
!pip install lpips

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.8/53.8 kB 5.5 MB/s eta 0:00:00


In [4]:
import io
import IPython.display
import cv2
import PIL.Image
import os
import numpy as np
from tqdm import tqdm
import torch
import torchvision.models as models
import torchvision.transforms as transforms
from torch.nn import functional as F
from torch.optim.lr_scheduler import StepLR
import torch.nn as nn
import lpips

from models.helper import build_generator
from utils.logger import setup_logger
from utils.editor import get_layerwise_manipulation_strength
from utils.editor import manipulate
from torchvision.transforms import ToTensor
from torchvision.models import vgg16, VGG16_Weights


# LPIPS 손실 초기화
# GPU가 사용 가능한 경우 CUDA를 사용하고, 그렇지 않으면 CPU를 사용하도록 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# LPIPS 손실 함수 초기화, 'alex' 네트워크를 사용
lpips_loss_fn = lpips.LPIPS(net='alex').to(device)

# Perceptual Loss(지각적 손실) 를 위해 VGG 초기화
# 사전 학습된 VGG16 네트워크의 첫 16개의 레이어만 사용하고 평가 모드로 설정
vgg = vgg16(weights="VGG16_Weights.IMAGENET1K_V1").features[:16].eval().to(device)
# VGG 네트워크의 매개변수(가중치)들이 학습되지 않도록 설정
for param in vgg.parameters():
    param.requires_grad = False

def imshow(images, col, viz_size=256):
    """하나의 그림에 여러 이미지를 표시"""
    num, height, width, channels = images.shape
    assert num % col == 0  # 열로 나누어 떨어지는지 확인
    row = num // col  # 행의 수 계산

    # 전체 이미지를 담을 빈 캔버스 생성
    fused_image = np.zeros((viz_size * row, viz_size * col, channels), dtype=np.uint8)

    for idx, image in enumerate(images):
        i, j = divmod(idx, col)  # 인덱스를 행, 열 위치로 변환
        y = i * viz_size  # y 축 위치 계산
        x = j * viz_size  # x 축 위치 계산
        if height != viz_size or width != viz_size:  # 이미지 크기가 지정한 시각화 크기와 다르면
            image = cv2.resize(image, (viz_size, viz_size))  # 이미지 크기 조정
        fused_image[y:y + viz_size, x:x + viz_size] = image  # 캔버스에 이미지를 붙임

    fused_image = np.asarray(fused_image, dtype=np.uint8)  # 정수형 배열로 변환
    data = io.BytesIO()  # 메모리 내 바이너리 데이터 버퍼 생성
    PIL.Image.fromarray(fused_image).save('test.jpeg', 'jpeg')  # 이미지를 JPEG 형식으로 저장
    im_data = data.getvalue()  # 바이너리 데이터로 변환
    disp = IPython.display.display(IPython.display.Image(im_data))  # 이미지 표시
    return disp

def build_model(model_name, logger=None):
    """모델 이름에 따라 생성기를 빌드합니다."""
    model = build_generator(model_name, logger=logger)  # 모델 빌드 함수 호출
    return model

def sample_codes(model, num, seed=0, w1k_code=None):
    """랜덤으로 잠재 코드를 샘플링합니다."""
    np.random.seed(seed)  # 난수 시드 설정
    if w1k_code is None:  # w1k 코드가 없는 경우
        latent_codes = model.easy_sample(num=num, latent_space_type='w')  # w 잠재 공간에서 샘플링
    else:  # w1k 코드가 있는 경우
        latent_codes = w1k_code[np.random.randint(0, w1k_code.shape[0], num)]  # w1k 코드에서 무작위로 선택
    # 샘플된 잠재 코드를 w 공간에 맞게 변환하고 스타일 및 이미지 생성 없이 반환
    latent_codes = model.easy_synthesize(latent_codes=latent_codes,
                                         latent_space_type='w',
                                         generate_style=False,
                                         generate_image=False)['wp']
    return latent_codes

def calculate_total_variation_loss(image):
    """이미지의 매끄러움을 위한 총 변동 손실(Total Variation Loss)"""
    tv_loss = torch.sum(torch.abs(image[:, :, :, :-1] - image[:, :, :, 1:])) + \
              torch.sum(torch.abs(image[:, :, :-1, :] - image[:, :, 1:, :]))  # 인접한 픽셀 간의 차이의 합계 계산
    return tv_loss

def perceptual_loss(input_image, generated_image):
    """VGG16을 사용하는 지각적 손실(Perceptual Loss)"""
    input_features = vgg(input_image)  # 입력 이미지의 특징 추출
    generated_features = vgg(generated_image)  # 생성된 이미지의 특징 추출
    return F.mse_loss(input_features, generated_features)  # 특징 간의 평균 제곱 오차(MSE) 계산

def color_loss(input_image, generated_image):
    """색상 일관성 손실(Color Consistency Loss)"""
    input_color = torch.mean(input_image, dim=(2, 3), keepdim=True)  # 입력 이미지의 평균 색상 계산
    generated_color = torch.mean(generated_image, dim=(2, 3), keepdim=True)  # 생성된 이미지의 평균 색상 계산
    return F.mse_loss(input_color, generated_color)  # 평균 색상 간의 MSE 손실 계산

def preprocess_image(image_path, device):
    """입력 이미지를 불러오고 전처리합니다."""
    if not os.path.exists(image_path):  # 이미지 경로가 존재하지 않으면 예외 발생
        raise FileNotFoundError(f"입력 이미지를 {image_path} 경로에서 찾을 수 없습니다.")
    image = PIL.Image.open(image_path).convert('RGB')  # 이미지를 열고 RGB로 변환
    transform = transforms.Compose([
        transforms.Resize((256, 256)),  # 256x256 크기로 조정
        transforms.ToTensor(),  # 이미지를 텐서로 변환
    ])
    image_tensor = transform(image).unsqueeze(0).to(device)  # 배치 차원을 추가하고 장치로 전송
    return image_tensor

def approximate_latent_code(image_path, model, num_steps=1500, lr=0.01, num_initializations=10):
    """인코더 없이 입력 이미지를 잠재 공간에서 근사(latent code)를 최적화"""
    input_tensor = preprocess_image(image_path, device)

    # 여러 초기화를 통해 잠재 코드를 초기화하고 최적의 결과를 추적
    best_latent_code = None
    best_loss = float('inf')
    best_image = None  # 가장 작은 손실 값을 가지는 이미지를 저장

    for initialization in range(num_initializations):
        # 잠재 공간에서 랜덤한 잠재 코드를 초기화 (평균 0.1)
        latent_code = (torch.randn(1, 14, 512, device=device) * 0.1).requires_grad_(True)

        # 옵티마이저와 학습률 스케줄러 설정
        optimizer = torch.optim.Adam([latent_code], lr=lr)
        scheduler = StepLR(optimizer, step_size=500, gamma=0.5)

        for step in range(num_steps):
            optimizer.zero_grad()  # 그래디언트 초기화

            # 잠재 코드를 기반으로 이미지를 생성
            latent_code_np = latent_code.detach().cpu().numpy()

            # 잠재 코드의 형태가 올바른지 확인하고 차원 확장
            if latent_code_np.ndim == 2:
                latent_code_np = latent_code_np[np.newaxis, ...]

            generated_data = model.easy_synthesize(latent_code_np, latent_space_type='wp')
            generated_image = generated_data['image']

            # 생성된 이미지를 텐서로 변환하여 학습 가능 상태로 만듦
            generated_image_tensor = torch.tensor(
                generated_image.transpose(0, 3, 1, 2), dtype=torch.float32, device=device
            ) / 255.0
            generated_image_tensor.requires_grad_()

            # 손실 계산
            mse_loss = F.mse_loss(generated_image_tensor, input_tensor) * 100.0  # MSE 손실
            tv_loss = calculate_total_variation_loss(generated_image_tensor) * 0.005  # TV 손실
            perceptual = perceptual_loss(input_tensor, generated_image_tensor) * 40.0  # Perceptual 손실
            lpips_loss = lpips_loss_fn(input_tensor, generated_image_tensor).mean() * 30.0  # LPIPS 손실

            # 총 손실
            total_loss = mse_loss + tv_loss + perceptual + lpips_loss

            # 역전파 수행 및 파라미터 업데이트
            total_loss.backward()
            optimizer.step()
            scheduler.step()

            # 중간 결과 로깅
            if step % 100 == 0:
                print(f"Initialization [{initialization + 1}/{num_initializations}], Step [{step}/{num_steps}], Total Loss: {total_loss.item()}")
                print(f"MSE Loss: {mse_loss.item()}, TV Loss: {tv_loss.item()}, Perceptual Loss: {perceptual.item()}, LPIPS Loss: {lpips_loss.item()}")

        # 초기화별 생성된 이미지를 저장
        generated_image_np = (generated_image_tensor[0].detach().cpu().numpy().transpose(1, 2, 0) * 255).clip(0, 255).astype(np.uint8)
        PIL.Image.fromarray(generated_image_np).save(f"generated_initialization_{initialization + 1}.jpeg")

        # 가장 작은 손실 값을 가진 잠재 코드와 이미지를 저장
        if total_loss.item() < best_loss:
            best_latent_code = latent_code.clone().detach()
            best_loss = total_loss.item()
            best_image = generated_image_np  # 손실 값이 가장 작은 이미지를 저장

    # 최적의 잠재 코드에서 이미지를 생성
    best_latent_code_np = best_latent_code.detach().cpu().numpy()

    # 잠재 코드의 형태가 올바른지 확인하고 차원 확장
    if best_latent_code_np.ndim == 2:
        best_latent_code_np = best_latent_code_np[np.newaxis, ...]

    final_data = model.easy_synthesize(best_latent_code_np, latent_space_type='wp')
    final_image = final_data['image']

    # 최종 이미지를 저장
    final_image_np = (final_image[0] * 255).clip(0, 255).astype(np.uint8)
    PIL.Image.fromarray(final_image_np).save("final_best_generated_image.jpeg")
    print("Final generated image saved as 'final_best_generated_image.jpeg'.")

    # 손실 값이 가장 작은 이미지를 저장
    if best_image is not None:
        PIL.Image.fromarray(best_image).save("best_image_with_min_loss.jpeg")
        print("Best image with minimum loss saved as 'best_image_with_min_loss.jpeg'.")

    return best_latent_code_np

w1k_code = np.load('order_w_1k.npy')  # 사전 학습된 잠재 코드(w1k 코드) 파일을 불러옵니다.

indoor_model_name = "stylegan_bedroom"  # 사용하려는 모델의 이름을 지정합니다.
indoor_model = build_model(indoor_model_name)  # 지정된 이름으로 모델을 빌드합니다.

# 입력 이미지 불러오기 및 잠재 코드로 변환하기
input_image_path = '/content/10_445_6.png'  # 입력 이미지의 경로를 설정합니다.
if os.path.exists(input_image_path):  # 입력 이미지 파일이 존재하는지 확인합니다.
  input_latent_code = approximate_latent_code(input_image_path, indoor_model)  # 입력 이미지를 잠재 코드로 근사합니다.
else:
  raise FileNotFoundError(f"Input image not found at {input_image_path}")  # 파일이 존재하지 않으면 예외를 발생시킵니다.

# 여러 샘플을 생성하고 싶다면, 잠재 코드를 복제하여 사용합니다.
num_samples = 8  # 생성할 샘플 수를 설정합니다.
indoor_latent_codes = np.tile(input_latent_code, (num_samples, 1, 1))  # 입력 잠재 코드를 복제합니다.

synthesis_kwargs = {'latent_space_type': 'wp'}  # 잠재 공간 유형을 지정합니다.

# 잠재 코드로부터 이미지를 생성합니다.
images = indoor_model.easy_synthesize(indoor_latent_codes, **synthesis_kwargs)['image']  # 잠재 코드로 이미지를 합성합니다.
imshow(images, col=num_samples)  # 생성된 이미지를 화면에 표시합니다.

#@title { display-mode: "form", run: "auto" }
attribute_name = 'indoor_lighting'  # 조작할 속성을 설정합니다. ['indoor_lighting', 'wood', 'cluttered_space', 'view'] 중 하나를 선택합니다.
path = f'boundaries/{indoor_model_name}/{attribute_name}_boundary.npy'  # 속성 경계 파일의 경로를 설정합니다.

try:
  boundary_file = np.load(path, allow_pickle=True).item()  # 경계 파일을 불러옵니다.
  boundary = boundary_file['boundary']  # 경계 데이터를 추출합니다.
  manipulate_layers = boundary_file['meta_data']['manipulate_layers']  # 조작할 레이어 정보를 추출합니다.
except ValueError:
  boundary = np.load(path)  # 경계 파일을 불러옵니다.
  if attribute_name == 'view':
    manipulate_layers = '0-4'  # 'view' 속성의 경우, 조작할 레이어를 설정합니다.
  else:
    manipulate_layers = '6-11'  # 기본 조작 레이어 설정입니다.

if attribute_name == 'view':
  strength = [1.0 for _ in range(indoor_model.num_layers)]  # 'view' 속성에 대한 강도를 설정합니다.
else:
  strength = get_layerwise_manipulation_strength(
    indoor_model.num_layers, indoor_model.truncation_psi, indoor_model.truncation_layers)  # 레이어별 조작 강도를 가져옵니다.

# 잠재 코드를 조작하여 새 이미지를 생성합니다.
distance = -3  #@param {type:"slider", min:-3.0, max:3.0, step:0.1}  # 속성 조작의 거리를 설정합니다.
indoor_codes = manipulate(latent_codes=indoor_latent_codes,
                     boundary=boundary,
                     start_distance=0,
                     end_distance=distance,
                     step=2,
                     layerwise_manipulation=True,
                     num_layers=indoor_model.num_layers,
                     manipulate_layers=manipulate_layers,
                     is_code_layerwise=True,
                     is_boundary_layerwise=False,
                     layerwise_manipulation_strength=strength)  # 조작 매개변수를 통해 잠재 코드를 조작합니다.

images = indoor_model.easy_synthesize(indoor_codes[:, 1], latent_space_type='wp')['image']  # 조작된 잠재 코드로 이미지를 합성합니다.
imshow(images, col=num_samples)  # 생성된 이미지를 화면에 표시합니다.

# 생성된 이미지를 저장합니다.
np.save('generated_images.npy', images)  # 생성된 이미지를 .npy 파일로 저장합니다.
PIL.Image.fromarray(images[0]).save('test.jpeg', 'jpeg')  # 첫 번째 이미지를 .jpeg 파일로 저장합니다.
np.save('latent_codes_1.npy', indoor_codes)  # 조작된 잠재 코드를 저장합니다.

# 잠재 코드를 다시 조작하여 속성을 더욱 변형합니다.
distance = 3  #@param {type:"slider", min:-3.0, max:3.0, step:0.1}  # 속성 조작의 거리를 설정합니다.
indoor_codes2 = manipulate(latent_codes=indoor_latent_codes,
                     boundary=boundary,
                     start_distance=0,
                     end_distance=distance,
                     step=2,
                     layerwise_manipulation=True,
                     num_layers=indoor_model.num_layers,
                     manipulate_layers=manipulate_layers,
                     is_code_layerwise=True,
                     is_boundary_layerwise=False,
                     layerwise_manipulation_strength=strength)  # 조작 매개변수를 통해 잠재 코드를 다시 조작합니다.

images2 = indoor_model.easy_synthesize(indoor_codes2[:, 1], latent_space_type='wp')['image']  # 새로 조작된 잠재 코드로 이미지를 합성합니다.
imshow(images2, col=num_samples)  # 생성된 이미지를 화면에 표시합니다.

# 새로 생성된 이미지를 저장합니다.
np.save('generated_images_2.npy', images2)  # 생성된 이미지를 .npy 파일로 저장합니다.
PIL.Image.fromarray(images2[0]).save('test_2.jpeg', 'jpeg')  # 첫 번째 이미지를 .jpeg 파일로 저장합니다.
np.save('latent_codes_2.npy', indoor_codes2)  # 새로 조작된 잠재 코드를 저장합니다.

# 원래 잠재 코드와 조작된 잠재 코드 사이의 보간 이미지를 생성합니다.
interpolated_images = []  # 보간 이미지를 저장할 리스트입니다.
steps = 10  # 보간 단계를 설정합니다.
for alpha in np.linspace(0, 1, steps):
    interpolated_code = (1 - alpha) * input_latent_code[0, :14, :] + alpha * indoor_codes2[0, :14, :]  # 0과 1 사이의 보간을 수행합니다.
    interpolated_images.append(indoor_model.easy_synthesize(interpolated_code, latent_space_type='wp')['image'][0])  # 보간된 코드로 이미지를 생성합니다.

interpolated_images = np.array(interpolated_images)  # 보간 이미지를 NumPy 배열로 변환합니다.
imshow(interpolated_images, col=steps)  # 보간 이미지를 화면에 표시합니다.

# 보간 이미지를 저장합니다.
np.save('interpolated_images.npy', interpolated_images)  # 보간 이미지를 .npy 파일로 저장합니다.
PIL.Image.fromarray(interpolated_images[0]).save('test_interpolated.jpeg', 'jpeg')  # 첫 번째 보간 이미지를 .jpeg 파일로 저장합니다.


Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/alexnet-owt-7be5be79.pth" to /root/.cache/torch/hub/checkpoints/alexnet-owt-7be5be79.pth
100%|██████████| 233M/233M [00:01<00:00, 227MB/s]


Loading model from: /usr/local/lib/python3.10/dist-packages/lpips/weights/v0.1/alex.pth


/usr/local/lib/python3.10/dist-packages/lpips/lpips.py:107: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.load_state_dict(torch.load(model_path, map_location='cpu'), st

Initialization [1/10], Step [0/1500], Total Loss: 305.3810119628906
MSE Loss: 7.234801292419434, TV Loss: 50.096805572509766, Perceptual Loss: 229.87298583984375, LPIPS Loss: 18.17643165588379
Initialization [1/10], Step [100/1500], Total Loss: 305.3810119628906
MSE Loss: 7.234801292419434, TV Loss: 50.096805572509766, Perceptual Loss: 229.87298583984375, LPIPS Loss: 18.17643165588379
Initialization [1/10], Step [200/1500], Total Loss: 305.3810119628906
MSE Loss: 7.234801292419434, TV Loss: 50.096805572509766, Perceptual Loss: 229.87298583984375, LPIPS Loss: 18.17643165588379
Initialization [1/10], Step [300/1500], Total Loss: 305.3810119628906
MSE Loss: 7.234801292419434, TV Loss: 50.096805572509766, Perceptual Loss: 229.87298583984375, LPIPS Loss: 18.17643165588379
Initialization [1/10], Step [400/1500], Total Loss: 305.3810119628906
MSE Loss: 7.234801292419434, TV Loss: 50.096805572509766, Perceptual Loss: 229.87298583984375, LPIPS Loss: 18.17643165588379
Initialization [1/10], Step